# Import dependencies

In [ ]:
import os
import sys

sys.path.insert(0, os.path.dirname(os.getcwd())) 

In [1]:
import time
import gc

import numpy as np
import pandas as pd

from transformers import BertTokenizer, BertModel
from transformers import logging

from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score

from semeval_reader import SemevalReader

from InputDataset import InputDataset

import torch
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

from torch import cuda

from models.BERT_Dropout_BiLSTM_Linear import BERT_Dropout_BiLSTM_Linear

In [2]:
device = 'cuda' if cuda.is_available() else 'cpu'

print(torch.cuda.get_device_name(0))
print(f"Memory: {torch.cuda.get_device_properties(0).total_memory // 1024 ** 3} GB")

NVIDIA GeForce RTX 2060 SUPER
Memory: 8 GB


In [3]:
logging.set_verbosity_error()

In [4]:
def clear_memory():
    torch.cuda.empty_cache()

    with torch.no_grad():
        torch.cuda.empty_cache()

    gc.collect()

# Load Data

In [5]:
def get_target_list_for_polarity(polarity):
    if polarity == 'positive':
        return [0, 0, 1]
    if polarity == 'negative':
        return [1, 0, 0]
    return [0, 1, 0]

In [6]:
semeval_reader = SemevalReader('../../../data/semeval16_restaurants_train.xml')

reviews = semeval_reader.read_reviews()
absolute_polarity_sentences = semeval_reader.get_absolute_polarity_sentences()

df = pd.DataFrame(map(lambda x: (x.text, x.opinions[0].polarity), absolute_polarity_sentences))
df.rename(columns={0: 'text'}, inplace=True)
df['target_list'] = df.apply(lambda row: get_target_list_for_polarity(row[1]), axis=1)

absolute_polarity_df = df.drop(columns=[1])

# Train & Validate

In [7]:
TRAIN_BATCH_SIZE = 4
VALID_BATCH_SIZE = 4

EPOCHS = 2

LEARNING_RATE = 1e-5

TRAIN_SPLIT = 0.8

NO_RUNS = 10

In [8]:
MODEL_OUTPUT = '../../../results/SA/SemEval16 - Task 5 - Restaurants/models/bert_pre_trained_dropout_bilstm_linear.pth'
STATS_OUTPUT = '../../../results/SA/SemEval16 - Task 5 - Restaurants/stats/bert_pre_trained_dropout_bilstm_linear.csv'

In [9]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [10]:
best_accuracy = 0.0

In [11]:
def train(epoch, model, loss_fn, optimizer, dataloader):
    model.train()

    dataloader_len = len(dataloader)

    for _,data in enumerate(dataloader, 0):
        optimizer.zero_grad()

        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)

        outputs = model(ids, mask, token_type_ids)

        loss = loss_fn(outputs, targets)
        
        if _ % (dataloader_len // 10) == 0:
            print(f"Epoch: {epoch}/{EPOCHS}, Batch: {_}/{dataloader_len}, Loss: {loss.item()}")
        
        loss.backward()
        
        optimizer.step()

In [12]:
def validation(model, dataloader):
    model.eval()
    
    fin_targets=[]
    fin_outputs=[]

    with torch.no_grad():
        for _, data in enumerate(dataloader, 0):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)

            outputs = model(ids, mask, token_type_ids)
            
            fin_targets.extend(targets.cpu().detach().numpy().tolist())
            fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())

    return fin_outputs, fin_targets

In [13]:
results = pd.DataFrame(columns=['accuracy','precision_score_micro','precision_score_macro','recall_score_micro','recall_score_macro','f1_score_micro','f1_score_macro', 'execution_time'])

In [14]:
for i in range(10):
    # clear cache cuda
    torch.cuda.empty_cache()
    with torch.no_grad():
        torch.cuda.empty_cache()
    gc.collect()

    start_time = time.time()

    print(f"Run {i + 1}/10")

    train_dataset = df.sample(frac=TRAIN_SPLIT)
    test_dataset = df.drop(train_dataset.index).reset_index(drop=True)
    train_dataset = train_dataset.reset_index(drop=True)

    training_set = InputDataset(train_dataset, tokenizer)
    testing_set = InputDataset(test_dataset, tokenizer)

    train_dataloader = DataLoader(
        training_set,
        sampler = RandomSampler(train_dataset),
        batch_size = TRAIN_BATCH_SIZE,
        drop_last = True
    )

    validation_dataloader = DataLoader(
        testing_set,
        sampler = SequentialSampler(testing_set),
        batch_size = VALID_BATCH_SIZE,
        drop_last = True
    )

    model = BERT_Dropout_BiLSTM_Linear(BertModel.from_pretrained('bert-base-uncased'), dropout=0.3, bilstm_in_features=256, no_out_labels=3, device=device).to(device)

    optimizer = torch.optim.Adam(params = model.parameters(), lr=LEARNING_RATE)
    loss_fn = torch.nn.BCEWithLogitsLoss()

    for epoch in range(EPOCHS):
        train(epoch, model, loss_fn, optimizer, train_dataloader)

    outputs, targets = validation(model, validation_dataloader)
    outputs = np.argmax(outputs, axis=1)
    targets = np.argmax(targets, axis=1)
    
    accuracy = accuracy_score(targets, outputs)
    precision_score_micro = precision_score(targets, outputs, average='micro')
    precision_score_macro = precision_score(targets, outputs, average='macro')
    recall_score_micro = recall_score(targets, outputs, average='micro')
    recall_score_macro = recall_score(targets, outputs, average='macro')
    f1_score_micro = f1_score(targets, outputs, average='micro')
    f1_score_macro = f1_score(targets, outputs, average='macro')

    execution_time = time.time() - start_time

    results.loc[i] = [accuracy,precision_score_micro,precision_score_macro,recall_score_micro,recall_score_macro,f1_score_micro,f1_score_macro, execution_time]

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        torch.save(model.bert, MODEL_OUTPUT)

    del train_dataset
    del test_dataset
    del training_set
    del testing_set
    del model
    del loss_fn
    del optimizer
    del outputs
    del targets

Run 1/10
Epoch: 0/2, Batch: 0/313, Loss: 0.7129162549972534
Epoch: 0/2, Batch: 31/313, Loss: 0.693769097328186
Epoch: 0/2, Batch: 62/313, Loss: 0.5946083664894104
Epoch: 0/2, Batch: 93/313, Loss: 0.5320090651512146
Epoch: 0/2, Batch: 124/313, Loss: 0.674825131893158
Epoch: 0/2, Batch: 155/313, Loss: 0.4543503522872925
Epoch: 0/2, Batch: 186/313, Loss: 0.4576895236968994
Epoch: 0/2, Batch: 217/313, Loss: 0.40240877866744995
Epoch: 0/2, Batch: 248/313, Loss: 0.5379345417022705
Epoch: 0/2, Batch: 279/313, Loss: 0.3476451337337494
Epoch: 0/2, Batch: 310/313, Loss: 0.6338309049606323
Epoch: 1/2, Batch: 0/313, Loss: 0.48881250619888306
Epoch: 1/2, Batch: 31/313, Loss: 0.3261891305446625
Epoch: 1/2, Batch: 62/313, Loss: 0.24056196212768555
Epoch: 1/2, Batch: 93/313, Loss: 0.2148626744747162
Epoch: 1/2, Batch: 124/313, Loss: 0.24718055129051208
Epoch: 1/2, Batch: 155/313, Loss: 0.3483841121196747
Epoch: 1/2, Batch: 186/313, Loss: 0.2935512065887451
Epoch: 1/2, Batch: 217/313, Loss: 0.190056487

D:\Apps\Anaconda\envs\ml\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Run 2/10
Epoch: 0/2, Batch: 0/313, Loss: 0.7138122320175171
Epoch: 0/2, Batch: 31/313, Loss: 0.6484948396682739
Epoch: 0/2, Batch: 62/313, Loss: 0.5635331869125366
Epoch: 0/2, Batch: 93/313, Loss: 0.6278156638145447
Epoch: 0/2, Batch: 124/313, Loss: 0.5604260563850403
Epoch: 0/2, Batch: 155/313, Loss: 0.5412669777870178
Epoch: 0/2, Batch: 186/313, Loss: 0.36972078680992126
Epoch: 0/2, Batch: 217/313, Loss: 0.3205537497997284
Epoch: 0/2, Batch: 248/313, Loss: 0.3346075415611267
Epoch: 0/2, Batch: 279/313, Loss: 0.30628424882888794
Epoch: 0/2, Batch: 310/313, Loss: 0.2880992293357849
Epoch: 1/2, Batch: 0/313, Loss: 0.3358098864555359
Epoch: 1/2, Batch: 31/313, Loss: 0.2683384120464325
Epoch: 1/2, Batch: 62/313, Loss: 0.24225696921348572
Epoch: 1/2, Batch: 93/313, Loss: 0.26103758811950684
Epoch: 1/2, Batch: 124/313, Loss: 0.23385944962501526
Epoch: 1/2, Batch: 155/313, Loss: 0.24995827674865723
Epoch: 1/2, Batch: 186/313, Loss: 0.20186758041381836
Epoch: 1/2, Batch: 217/313, Loss: 0.1319

D:\Apps\Anaconda\envs\ml\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Run 3/10
Epoch: 0/2, Batch: 0/313, Loss: 0.6960453987121582
Epoch: 0/2, Batch: 31/313, Loss: 0.6751636266708374
Epoch: 0/2, Batch: 62/313, Loss: 0.5906842947006226
Epoch: 0/2, Batch: 93/313, Loss: 0.4928397536277771
Epoch: 0/2, Batch: 124/313, Loss: 0.6866273880004883
Epoch: 0/2, Batch: 155/313, Loss: 0.3944697976112366
Epoch: 0/2, Batch: 186/313, Loss: 0.37693914771080017
Epoch: 0/2, Batch: 217/313, Loss: 0.38850268721580505
Epoch: 0/2, Batch: 248/313, Loss: 0.26584458351135254
Epoch: 0/2, Batch: 279/313, Loss: 0.3993379473686218
Epoch: 0/2, Batch: 310/313, Loss: 0.32272472977638245
Epoch: 1/2, Batch: 0/313, Loss: 0.31503230333328247
Epoch: 1/2, Batch: 31/313, Loss: 0.315265417098999
Epoch: 1/2, Batch: 62/313, Loss: 0.22991949319839478
Epoch: 1/2, Batch: 93/313, Loss: 0.21430718898773193
Epoch: 1/2, Batch: 124/313, Loss: 0.20036667585372925
Epoch: 1/2, Batch: 155/313, Loss: 0.2534189522266388
Epoch: 1/2, Batch: 186/313, Loss: 0.1655375212430954
Epoch: 1/2, Batch: 217/313, Loss: 0.4545

...

D:\Apps\Anaconda\envs\ml\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Run 9/10
Epoch: 0/2, Batch: 0/313, Loss: 0.7151336073875427
Epoch: 0/2, Batch: 31/313, Loss: 0.6303499937057495
Epoch: 0/2, Batch: 62/313, Loss: 0.5234854221343994
Epoch: 0/2, Batch: 93/313, Loss: 0.5366374254226685
Epoch: 0/2, Batch: 124/313, Loss: 0.5724115371704102
Epoch: 0/2, Batch: 155/313, Loss: 0.4801720976829529
Epoch: 0/2, Batch: 186/313, Loss: 0.5792274475097656
Epoch: 0/2, Batch: 217/313, Loss: 0.5083258152008057
Epoch: 0/2, Batch: 248/313, Loss: 0.42614269256591797
Epoch: 0/2, Batch: 279/313, Loss: 0.28371211886405945
Epoch: 0/2, Batch: 310/313, Loss: 0.24209213256835938
Epoch: 1/2, Batch: 0/313, Loss: 0.45691347122192383
Epoch: 1/2, Batch: 31/313, Loss: 0.34864962100982666
Epoch: 1/2, Batch: 62/313, Loss: 0.2581982910633087
Epoch: 1/2, Batch: 93/313, Loss: 0.4940466284751892
Epoch: 1/2, Batch: 124/313, Loss: 0.1796644628047943
Epoch: 1/2, Batch: 155/313, Loss: 0.22427880764007568
Epoch: 1/2, Batch: 186/313, Loss: 0.25714537501335144
Epoch: 1/2, Batch: 217/313, Loss: 0.1715

D:\Apps\Anaconda\envs\ml\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Run 10/10
Epoch: 0/2, Batch: 0/313, Loss: 0.7015601992607117
Epoch: 0/2, Batch: 31/313, Loss: 0.6669694185256958
Epoch: 0/2, Batch: 62/313, Loss: 0.6133999228477478
Epoch: 0/2, Batch: 93/313, Loss: 0.5895928144454956
Epoch: 0/2, Batch: 124/313, Loss: 0.6310868263244629
Epoch: 0/2, Batch: 155/313, Loss: 0.4853489100933075
Epoch: 0/2, Batch: 186/313, Loss: 0.4425649344921112
Epoch: 0/2, Batch: 217/313, Loss: 0.39546728134155273
Epoch: 0/2, Batch: 248/313, Loss: 0.4167359471321106
Epoch: 0/2, Batch: 279/313, Loss: 0.34983426332473755
Epoch: 0/2, Batch: 310/313, Loss: 0.3302704095840454
Epoch: 1/2, Batch: 0/313, Loss: 0.3523334264755249
Epoch: 1/2, Batch: 31/313, Loss: 0.3437492251396179
Epoch: 1/2, Batch: 62/313, Loss: 0.19927631318569183
Epoch: 1/2, Batch: 93/313, Loss: 0.2273930162191391
Epoch: 1/2, Batch: 124/313, Loss: 0.33480304479599
Epoch: 1/2, Batch: 155/313, Loss: 0.27368399500846863
Epoch: 1/2, Batch: 186/313, Loss: 0.20710329711437225
Epoch: 1/2, Batch: 217/313, Loss: 0.2245365

D:\Apps\Anaconda\envs\ml\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [15]:
results

,accuracy,precision_score_micro,precision_score_macro,recall_score_micro,recall_score_macro,f1_score_micro,f1_score_macro,execution_time
0,0.871795,0.871795,0.563333,0.871795,0.605282,0.871795,0.581874,208.818245
1,0.875000,0.875000,0.590317,0.875000,0.572048,0.875000,0.577543,202.117968
2,0.897436,0.897436,0.595549,0.897436,0.601357,0.897436,0.597918,199.705913
3,0.891026,0.891026,0.584011,0.891026,0.615590,0.891026,0.597078,196.075708
4,0.916667,0.916667,0.600749,0.916667,0.615781,0.916667,0.608134,193.086529
5,0.884615,0.884615,0.568846,0.884615,0.612281,0.884615,0.585789,190.209978
6,0.900641,0.900641,0.590639,0.900641,0.603471,0.900641,0.596923,206.704926
7,0.865385,0.865385,0.584871,0.865385,0.573333,0.865385,0.574938,199.224072
8,0.878205,0.878205,0.573234,0.878205,0.605059,0.878205,0.588403,198.574324
9,0.891026,0.891026,0.579315,0.891026,0.597363,0.891026,0.588175,201.510039


In [16]:
results.to_csv(STATS_OUTPUT)